In [87]:
import statistics
from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from collections import Counter
from math import sqrt, ceil

In [88]:
wb = load_workbook('data\\someData.xlsx')
ws = wb.active

In [89]:
watering_values = []
for row in ws.iter_rows(min_row=2, min_col=3, max_col=3, values_only=True):
    watering = row[0]
    if isinstance(watering, (int, float)):
        watering_values.append(int(watering))

In [90]:
counts = Counter(watering_values)

In [91]:
new_ws = wb.create_sheet(title="Discrete Series")

In [92]:
new_ws.append(["Watering", "Number of Farms", "xf"])

In [93]:
total_farms = 0
total_xf = 0

In [94]:
for watering in range(2, 11):
    num_farms = counts.get(watering, 0)
    xf = watering * num_farms
    new_ws.append([watering, num_farms, xf])
    
    total_farms += num_farms
    total_xf += xf

In [95]:
new_ws.append(["Total", total_farms, total_xf])

In [96]:
new_ws.append([])

In [97]:
median = statistics.median(watering_values)
mean = total_xf / total_farms 
mode = statistics.mode(watering_values)
new_ws.append(['mode', 'median', 'mean'])
new_ws.append([mode, median, mean])

In [98]:
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

In [99]:
for row in new_ws.iter_rows(min_row=1, max_row=11, min_col=1, max_col=3):
    for cell in row:
        cell.border = thin_border

for row in new_ws.iter_rows(min_row=13, max_row=14, min_col=1, max_col=3):
    for cell in row:
        cell.border = thin_border

In [100]:
new_ws_2 = wb.create_sheet(title="Rank-size Series")
new_ws_2.append(["Number", "Cabbage yield, t/ha"])

In [101]:
data = []
for row in ws.iter_rows(min_row=2, min_col=1, max_col=2, values_only=True):
    number = row[0]
    crops = row[1]
    data.append([number, crops])

In [102]:
sorted_data = sorted(data, key=lambda x: x[1])

for row in sorted_data:
    new_ws_2.append(row)

In [103]:
sumCrops = sum(row[1] for row in sorted_data if row[1] is not None) 

In [104]:
new_ws_2.append(["Sum", sumCrops])

In [105]:
new_ws_2.append([])
new_ws_2.append([])

In [106]:
medianNew = statistics.median(row[1] for row in sorted_data if row[1] is not None)
meanNew = statistics.mean(row[1] for row in sorted_data if row[1] is not None)
modeNew = statistics.mode(row[1] for row in sorted_data if row[1] is not None)
new_ws_2.append(['mode', 'median', 'mean'])
new_ws_2.append([modeNew, medianNew, meanNew])

In [107]:
for row in new_ws_2.iter_rows(min_row=1, max_row=32, min_col=1, max_col=2):
    for cell in row:
        cell.border = thin_border

for row in new_ws_2.iter_rows(min_row=35, max_row=36, min_col=1, max_col=3):
    for cell in row:
        cell.border = thin_border

In [108]:
new_ws_3 = wb.create_sheet(title="TBD Series")
new_ws_3.append(["Interval", "Lower Bound", "Upper Bound", "Number of Farms"])

In [109]:
sqrtTotalFarms = ceil(sqrt(total_farms))

column_letter = 'B'
cells = ws[column_letter] 
values = []
for cell in cells:
    if cell.row == 1:
        continue
    if isinstance(cell.value, (int, float)):
        values.append(cell.value)

maxCrop = max(values)
minCrop = min(values)

hStep = (maxCrop - minCrop) / sqrtTotalFarms

In [110]:
temp = 1
lBound = minCrop
freq = 0
tempSumForTable = 0
for row in new_ws_3.iter_rows(min_row=1, max_row=sqrtTotalFarms, min_col=1, max_col=4, values_only=True):
    uBound = lBound + hStep
    if temp == 6:
        count = sum(lBound <= val <= ceil(uBound) for val in values)
    else:
        count = sum(lBound <= val <= uBound for val in values)
    avg = (lBound + uBound) / 2
    freq += count
    tempSumForTable += (count * avg)
    new_ws_3.append([temp, lBound, uBound, count, '', avg, avg*count, freq])
    lBound = uBound
    temp += 1

In [111]:
new_ws_3.append([])
new_ws_3.append(['min', 'max', 'step', 'sqrt of total farms'])
new_ws_3.append([minCrop, maxCrop, hStep, sqrtTotalFarms])
new_ws_3.append([])

In [112]:
new_ws_3['F1'] = 'Average'
new_ws_3['G1'] = 'Multiplication'
new_ws_3['H1'] = 'Summaries'


In [113]:
interval_data = []
for row in new_ws_3.iter_rows(min_row=2, values_only=True):
    if row[0] is None or not isinstance(row[0], int):
        break
    interval_data.append(row)
    
counts = [row[3] for row in interval_data]
max_count = max(counts)
mode_index = counts.index(max_count)
mode_lower = interval_data[mode_index][1]
mode_upper = interval_data[mode_index][2]
modeTBD = f"{mode_lower} - {mode_upper}"

median_pos = total_farms / 2
cumulative_freqs = [row[7] for row in interval_data]

for i, cf in enumerate(cumulative_freqs):
    if cf >= median_pos:
        median_class = interval_data[i]
        break

L = median_class[1]
h = hStep
F_prev = cumulative_freqs[i-1] if i > 0 else 0
f = median_class[3]

median_value = L + ((median_pos - F_prev) / f) * h
medianTBD = median_value
meanTBD = tempSumForTable / total_farms

In [114]:
new_ws_3['F9'] = 'Mode'
new_ws_3['G9'] = 'Mean'
new_ws_3['H9'] = 'Median'
new_ws_3['F10'] = modeTBD
new_ws_3['G10'] = meanTBD
new_ws_3['H10'] = medianTBD

In [115]:
for row in new_ws_3.iter_rows(min_row=1, max_row=7, min_col=1, max_col=4):
    for cell in row:
        cell.border = thin_border
        
for row in new_ws_3.iter_rows(min_row=1, max_row=7, min_col=6, max_col=8):
    for cell in row:
        cell.border = thin_border

for row in new_ws_3.iter_rows(min_row=9, max_row=10, min_col=1, max_col=4):
    for cell in row:
        cell.border = thin_border

for row in new_ws_3.iter_rows(min_row=9, max_row=10, min_col=6, max_col=8):
    for cell in row:
        cell.border = thin_border

In [116]:
wb.save('output.xlsx')